In [3]:
import numpy as np
import const
from fpath import *
from scipy.interpolate import interp1d
from scipy.interpolate import RegularGridInterpolator
from scipy.interpolate import griddata
from math import pi, log10, sqrt, log, exp, sqrt, atan, cos, sin, acos, asin

In [4]:
# adjustable parameters
nH0 = 1                # [cm^-3] H number density
tmax = 300.     # [s], duration of the source
LUV = 3e47      # [erg/s]  # source luminosity

# fixed parameters
tmin = 0.
n0_over_nH = 1.45e-15    # dust number density over H number density
lam0 = 2.       # [um] critical wavelength for Qabs_lambda
thej = pi/2     # [rad] jet opening angle
p = 2.2         # electron PL index => spectrum L_nu ~ nu^{(1-p)/2}
nuUVmax = 50/const.erg2eV/const.H_PLANCK   # maximum UV frequency 50 eV
LnuUVmax = (3-p)/2*LUV/nuUVmax   # Lnu at nuUVmax

In [5]:
amin, amax = 0.01, 0.3      # um, grain size limits
Na = 30
aarr = np.logspace(log10(amin), log10(amax), Na)
a_ratio = aarr[1]/aarr[0]

rmin, rmax = 0.4, 100.       # pc, radial layers
Nr = 150     # we need dr/r <~ thej to resolve the light curve of the echo
rarr = np.logspace(log10(rmin), log10(rmax), Nr)
r_ratio = rarr[1]/rarr[0]

#THIS IS NEW
themin, themax = 0.000001, pi/2    #theta array
Nthe = 50
thearr = np.linspace(themin, themax, Nthe)
Dthe = thearr[1]-thearr[0]

# min and max source frequencies
numin, numax = 0.1/(const.erg2eV*const.H_PLANCK), 50/(const.erg2eV*const.H_PLANCK)
Nnu = 40
nuarr = np.logspace(log10(numin), log10(numax), Nnu)    # frequency bins
nu_ratio = nuarr[1]/nuarr[0]

# jet emission time [dust local frame]
Nt = 20     # this is the dimension we interpolate over
tarr = np.linspace(tmin, tmax, Nt, endpoint=False)
dt = tarr[1] - tarr[0]
tarr += dt/2.

In [6]:
Tarr = np.zeros((Nt, Nr, Na, Nthe), dtype=float)   # to store the dust temperature
asubarr = np.zeros((Nt, Nr, Nthe), dtype=float)    # to store sublimation radii
taudarr = np.zeros((Nnu, Nr, Nthe), dtype=float)   # dust extinction optical depth at each nu
jdnuarr = np.zeros((Nt, Nr, Nthe), dtype=float)    # volumetric emissivity at lamobs

In [7]:
def func_Lnu(t, nu):     # source spectrum and light curve
    if t < tmax:
        return LnuUVmax*(nu/nuUVmax)**((1 - p)/2)  # spectrum L_nu ~ nu^{(1-p)/2}
    return 0.


def func_nH(r, the):      # gas density profile (r in pc)
    return nH0


def func_Qabs(nu, a):      # absorption efficiency for grain size a [um]
    lam = const.C_LIGHT/nu * 1e4    # wavelength in um
    return 1./(1 + (lam/lam0)**2 / a)


def func_T(qdot_h, aum):    # solve the heating and cooling balance for T
    y = qdot_h/(7.12576*aum**2)
    if y >= (31.5/aum)**2/12:
        return 3240/sqrt(aum)   # the grain should evaporate immediately
    xi = sqrt((31.5/aum)**2 - 12*y) + 31.5/aum
    T3 = sqrt((2*y*y/3/xi)**(1./3) + (xi*y/18)**(1./3))
    return 1000*T3

In [9]:
# compute cumulative number of H as a function of radius and angle
Nr_fine = int(Nr*10)
Nthe_fine = int(Nthe*10)
rarr_fine = np.logspace(log10(rmin/10), log10(rmax), Nr_fine)
thearr_fine = np.linspace(themin, themax, Nthe_fine)
r_ratio_fine = rarr_fine[1]/rarr_fine[0]
Dthe_fine = thearr_fine[1]-thearr_fine[0]
NHarr_fine = np.zeros((Nthe_fine, Nr_fine), dtype=float)
interp_fns = np.zeros(Nthe_fine, dtype=object)
NH = 0.

In [10]:
for i in range(Nthe_fine):
    NH = 0
    for j in range(Nr_fine):
        r = rarr_fine[j]
        the = thearr_fine[i]
        dr = r * (sqrt(r_ratio_fine) - 1/sqrt(r_ratio_fine))
        NH += 2*pi * r*r*np.sin(the)*Dthe_fine*dr*const.pc2cm**3 * func_nH(r, the)
        NHarr_fine[i][j] = NH
    rthe_NH_intp = interp1d(rarr_fine, NHarr_fine[i], fill_value='extrapolate')
    interp_fns[i] = rthe_NH_intp

In [11]:
sin_the = np.sin(thearr_fine)
NH_matrix = 2 * pi * (rarr_fine ** 2)[:, None] * sin_the[None, :] * const.pc2cm ** 3 * func_nH(
    rarr_fine[:, None], thearr_fine[None, :])
dr_matrix = rarr_fine[:, None] * (np.sqrt(r_ratio_fine) - 1 / np.sqrt(r_ratio_fine))
NHarr_fine_vec = np.cumsum(NH_matrix * dr_matrix, axis=0)

In [13]:
interp_fns_vec = np.zeros(Nthe_fine, dtype=object)

for i in range(Nthe_fine):
    interp_fns_vec[i] = interp1d(rarr_fine, NHarr_fine_vec[:, i], fill_value='extrapolate')

In [20]:
NHarr_fine

array([[1.94115760e+41, 3.91295010e+41, 5.91586097e+41, ...,
        1.89201575e+53, 1.92187511e+53, 1.95220569e+53],
       [6.11247472e+44, 1.23214151e+45, 1.86283435e+45, ...,
        5.95773287e+56, 6.05175643e+56, 6.14726385e+56],
       [1.22229477e+45, 2.46387951e+45, 3.72505866e+45, ...,
        1.19135147e+57, 1.21015310e+57, 1.22925146e+57],
       ...,
       [1.94111913e+47, 3.91287255e+47, 5.91574373e+47, ...,
        1.89197826e+59, 1.92183702e+59, 1.95216700e+59],
       [1.94114798e+47, 3.91293071e+47, 5.91583166e+47, ...,
        1.89200638e+59, 1.92186558e+59, 1.95219602e+59],
       [1.94115760e+47, 3.91295010e+47, 5.91586097e+47, ...,
        1.89201575e+59, 1.92187511e+59, 1.95220569e+59]])

In [19]:
NHarr_fine_vec.T

array([[6.16654268e+43, 1.24304043e+44, 1.87931207e+44, ...,
        6.01043207e+55, 6.10528731e+55, 6.20163954e+55],
       [1.94177105e+47, 3.91418667e+47, 5.91773050e+47, ...,
        1.89261367e+59, 1.92248246e+59, 1.95282263e+59],
       [3.88290620e+47, 7.82709152e+47, 1.18335231e+48, ...,
        3.78460754e+59, 3.84433534e+59, 3.90500575e+59],
       ...,
       [6.16642047e+49, 1.24301580e+50, 1.87927483e+50, ...,
        6.01031295e+61, 6.10516631e+61, 6.20151663e+61],
       [6.16651213e+49, 1.24303427e+50, 1.87930276e+50, ...,
        6.01040229e+61, 6.10525706e+61, 6.20160881e+61],
       [6.16654268e+49, 1.24304043e+50, 1.87931207e+50, ...,
        6.01043207e+61, 6.10528731e+61, 6.20163954e+61]])

In [22]:
interp_fns_vec.shape

(500,)

In [23]:
interp_fns.shape

(500,)

In [24]:
interp_fns==interp_fns_vec

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,